### KoGPT-2를 활용한 Few-shot & Zero-shot 실습Permalink
Transformers를 설치하고 Model을 load하겠습니다

In [5]:
!pip install transformers

In [4]:
# 모델 다운로드 받고
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.12
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Need to get 7,420 kB of archives.
After this operation, 6,051 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu jammy/main amd64 git-lfs amd64 3.5.1 [7,420 kB]
Fetched 7,420 kB in 0s (23.9 MB/s)
(Reading database ... 121757 files and directories c

In [7]:
# 모델 로드하고
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
config.pad_token_id = tokenizer.token_to_id('<pad>')
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cpu'), strict=False) # cuda대신에 cpu

_IncompatibleKeys(missing_keys=[], unexpected_keys=['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias'])

In [8]:
tokenized_text = tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

# 토크나이징 테스트 해보고
# 원하는대로 토크나이징이 잘 되고 vocab_id 도 반환하는 것을 확인할 수 있음

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['▁이순', '신은', '▁조선', '▁중기의', '▁무신', '이다', '.']
[10925, 6647, 1117, 40249, 39793, 128, 47440]


In [9]:
tokenizer.add_special_tokens(["<s>", "</s>"])
# 명시적으로 문장의 시작과 끝에는 ["<s>", "</s>"] 이런 기호가 들어가는 것을 tokenizer 에 지정해줌

2

In [12]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("이순신은", add_special_tokens=True).ids).unsqueeze(0).to('cpu')

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

# 생성도 테스트 해봄
# 자연스럽게 잘 생성되는 것을 볼 수 있음

GENERATED SEQUENCE : 이순신은 “그 동안 은녀로 계셨던 여러분과 함께 하고 싶었습니다. 두 사람 모두 30년 이상의 긴 호흡을 맞춘 첫사랑이라고. 이들은 서로의 마음을 확인하는 '의자놀이'를 하며 서로를 달랬다. 두 사람은 "내가 먼저 미안해 하는 것 같아 미안하다. 하지만, 좋은 남자로 시작해 보자는 뜻에서 (유)남편의 제안을 받아들였으니 결혼해서 잘 살자는 바람"이라고 말하며 진심 어린 소감을 전했다.
GENERATED SEQUENCE : 이순신은 자신의 아들이자 동무인 최숙향(김향수 분)를 만나러 가지만 아들 최충헌의 죽음과 함께 동무들 앞에서 죽음을 맞이하게 된다. 한재석 측 "유리한 해명 불구 해명글 게재는 아냐" 사진퍼가기 이용안내 배우 한재석이 연예계 대표 ‘남남갈등’의 원인을 밝혔다. 그는 앞서 KBS 2FM 라디오 '오후의 발견(이하 '오후의 발견')'
GENERATED SEQUENCE : 이순신은 "내가 나 때문에 내 아들을 잃은 건 아닐까?" 이 때문에 박 씨는 '오블리비'(오블리비언을 위한 몸값)를 받으려고 한 것으로 보였다. 제작진은 "그녀가 오블리비언을 통해 자신의 삶을 보았을 때 자신의 딸들이 어떤 삶을 산다고 믿었을까도 생각한다"고 말했다. "오블리비언을 하면서 딸이 스스로 살면 되는 것 아닌가"라며 안도의 한숨을 내쉰 박 씨는 "딸의 이야기를 하면서


In [15]:
# top_k, top_p, do_sample 이런 것들을 넣어줬음

def get_gpt_output(input_sent):
    input_ids = torch.tensor(tokenizer.encode(input_sent, add_special_tokens=True).ids).unsqueeze(0).to('cpu')
    sample_outputs = model.generate(
        input_ids,
        do_sample=True,
        max_length=512,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.token_to_id("</s>"),     #generate 옵션에서 줄 수 있는게 eos_token_id 옵션이 있음
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_sequence = sample_outputs[0].tolist()
    return tokenizer.decode(generated_sequence, skip_special_tokens=True)


# eos_token_id=tokenizer.token_to_id("</s>") 이렇게 지정해주면 나중에 문장을 생성하다가 내가 지정한 토큰이 등장하게되면 early_stopping 이 가능해지게 됨
# 그 이상 생성하지 않게 됨

In [16]:
get_gpt_output("<s>이순신은")
# 입력으로 “<s>이순신은” 이라는 문장을 주었더니
# 문장 마침 기호가 등장했기 때문에 더이상 생성하지 않고 early stopping 이 된 것임

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


'이순신은 또 “난 이제 늙은이구나.'

이 방법을 이용해서 Zero-shot learning 을 해보자

In [17]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : ")
# 대화체가 나오길 기대했는데 “….” 이 나옴

'철수 : 영희야 안녕! 영희 : ....'

이번에는 Few-shot learning 을 해보자

In [25]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : 어! 철수야! 오랜만이다!</s><s>철수 : 그러게~ 잘 지냈어?</s><s>영희 : ")
# 이상한 말

'철수 : 영희야 안녕! 영희 : 어! 철수야! 오랜만이다! 철수 : 그러게~ 잘 지냈어? 영희 : 네...아아, 아...휴...나 좀 자요...헤헤...엄마'

One-shot learning 으로 감정에 대한 분류를 해보자

In [24]:
get_gpt_output("<s>본문 : 아.. 기분 진짜 짜증나네ㅡㅡ</s><s>감정 : 분노</s><s>본문 : 와!! 진짜 너무 좋아!!</s><s>감정 : ")

'본문 : 아.. 기분 진짜 짜증나네 감정 : 분노 본문 : 와!! 진짜 너무 좋아!! 감정 : 짱이야!!...'

이번에는 open domain question 을 해봄

Few-shot learning 으로 진행

In [26]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 인공지능의 미래에 대해 어떻게 생각하십니까?</s>\
<s>답 : ")

# 답변의 연관성이 있다..-> 있나?

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 인공지능의 미래에 대해 어떻게 생각하십니까? 답 : \\1부터 5까지 3단계 언어를 인식할 수 있는 과 2단계 언어의 사용을 기억할수 있는 언어 ap -> 그리고 3 단계의 언어적 지식은 op을 통해 인지되는 것으로 보입니다. ad) (b) @FF 제8부 1.'

In [28]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 자연어처리에서 언어모델이란 무엇입니까?</s>\
<s>답 : ")

# 맞는말인지는 잘 모르겠지만 기존의 자연어를 생성하던 패턴이랑은 굉장히 다른 양상을 보임

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 자연어처리에서 언어모델이란 무엇입니까? 답 : rank(스몰캡) 에서 시작하여 STEA, Smith, Gardens, Johnson, Bock, Jr., Phonkia, Stardu, Preach, Alexander, Alco, Cataril, Lanquer., Khing, Nilfel, Repith)에서 시작된다. seffected는 것은 언어 모델의 오류 또는 오류의 영향을 받는 것을 뜻합니다. culture palteriority가 있습니다 : courally castology의 개념에 대한 제안; schemberness 의 의미 (

In [29]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 이순신 장군이 전사한 전투는 무슨 전투입니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 이순신 장군이 전사한 전투는 무슨 전투입니까? 답 : -> ~ ～> (순창고신 전투) (신선 전투):(순창 전투 : 전투);> -(11) 무영영(무영 전투)-> 무정(민 전투 - 전투)(상) 무(우영전투): (무정 전투),<<무무(중 무무전투 : 무중 전투)> * 아리스샘터 = (최정전투) 전쟁 - <무당 전투> * 마왕: <영사기(영사사기) [무법자백):> <하>(가운:> ) * 야야(야야 전투 전투-) * 하(하) > * (도트: 무적자군) 전투: * 오(오): 가운) 전투에 필요한 \'무기\'들\' * 마법사: \'비공식\' 전투에 

In [30]:
# 번역
get_gpt_output("<s>한국어: 그 도로는 강과 평행으로 뻗어 있다.</s>\
<s>English: The road runs parallel to the river.</s>\
<s>한국어: 그 평행선들은 분기하는 것처럼 보인다.</s>\
<s>English: The parallel lines appear to diverge.</s>\
<s>한국어: 그 도로와 운하는 서로 평행하다.</s>\
<s>English: The road and the canal are parallel to each other.</s>\
<s>한국어: 평행한 은하계라는 개념은 이해하기가 힘들다.</s>\
<s>English: The idea of a parallel universe is hard to grasp.</s>\
<s>한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다.</s>\
<s>English: This tradition has no parallel in our culture.</s>\
<s>한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다.</s>\
<s>English: This is an achievement without parallel in modern times.</s>\
<s>한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다.</s>\
<s>English: It is possible to draw a parallel between their experience and ours.</s>\
<s>한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다.</s>\
<s>English: The new degree and the existing certificate courses would run in parallel.</s>\
<s>한국어: 이순신은 조선 중기의 무신이다.</s>\
<s>Englisth: ")

'한국어: 그 도로는 강과 평행으로 뻗어 있다. English: The road runs parallel to the river. 한국어: 그 평행선들은 분기하는 것처럼 보인다. English: The parallel lines appear to diverge. 한국어: 그 도로와 운하는 서로 평행하다. English: The road and the canal are parallel to each other. 한국어: 평행한 은하계라는 개념은 이해하기가 힘들다. English: The idea of a parallel universe is hard to grasp. 한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다. English: This tradition has no parallel in our culture. 한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다. English: This is an achievement without parallel in modern times. 한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다. English: It is possible to draw a parallel between their experience and ours. 한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다. English: The new degree and the existing certificate courses would run in parallel. 한국어: 이순신은 조선 중기의 무신이다. Englisth: unregard for every sale. plain science (주: Auter, the same) 한글: 한글과 그 name(이름): [The channer for open the best underful. serviced and individual system 한글로 쓴 organ of othlves) 문법: Engly, metal, wide 한글 뜻

In [31]:
get_gpt_output("<s>한국어: 그 도로는 강과 평행으로 뻗어 있다.</s>\
<s>English: The road runs parallel to the river.</s>\
<s>한국어: 그 평행선들은 분기하는 것처럼 보인다.</s>\
<s>English: The parallel lines appear to diverge.</s>\
<s>한국어: 그 도로와 운하는 서로 평행하다.</s>\
<s>English: The road and the canal are parallel to each other.</s>\
<s>한국어: 평행한 은하계라는 개념은 이해하기가 힘들다.</s>\
<s>English: The idea of a parallel universe is hard to grasp.</s>\
<s>한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다.</s>\
<s>English: This tradition has no parallel in our culture.</s>\
<s>한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다.</s>\
<s>English: This is an achievement without parallel in modern times.</s>\
<s>한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다.</s>\
<s>English: It is possible to draw a parallel between their experience and ours.</s>\
<s>한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다.</s>\
<s>English: The new degree and the existing certificate courses would run in parallel.</s>\
<s>한국어: 100명은 서 있을 수 있을 것 같은 섬 하나가 넓은 하늘 위에 두둥실 떠 있다.</s>\
<s>Englisth: ")

'한국어: 그 도로는 강과 평행으로 뻗어 있다. English: The road runs parallel to the river. 한국어: 그 평행선들은 분기하는 것처럼 보인다. English: The parallel lines appear to diverge. 한국어: 그 도로와 운하는 서로 평행하다. English: The road and the canal are parallel to each other. 한국어: 평행한 은하계라는 개념은 이해하기가 힘들다. English: The idea of a parallel universe is hard to grasp. 한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다. English: This tradition has no parallel in our culture. 한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다. English: This is an achievement without parallel in modern times. 한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다. English: It is possible to draw a parallel between their experience and ours. 한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다. English: The new degree and the existing certificate courses would run in parallel. 한국어: 100명은 서 있을 수 있을 것 같은 섬 하나가 넓은 하늘 위에 두둥실 떠 있다. Englisth: all style is cross, at leaving.com quallier. must remind ek. preach in horderly happened by a gradior of half, including, when eased record. business speech agendarises bills,

KoGPT-2 를 활용해도 Zero-shot, One-shot, Few-shot leanring 을 할 수 있음을 알 수 있음

다음 실습에서는 원하는 답변을 내놓도록 GPT-2 를 fine-tuning 해보자